구글브레인의 Hinton교수가 이번에 Dynamic Routing Between Capsules 페이퍼를 공개했습니다.<br>
기존 CNN이 갖고 있는 문제를 해결했다고 하는데 해당 문서에서는 Capsule Network가 무엇으며, 기존 CNN과의 차이가 무엇인지 설명합니다.

# Convolutional Neural Network

1998년 얀레쿤 (Yann LeCun) 교수는 AT&T Bell Laboratories에 합류하게 되고, 이곳에서 여러가지 기계학습 기법들을 만들게 됩니다. <br>
이미지 인식에서 사용하던 convolution에 영감을 받아서 Convolutional Neural Network라는 모델을 만들게 되었고, 현재까지 이미지 인식 분야에서 높은 인식률을 보여주고 있습니다. 

CNN에 대한 자세한 설명은 다음을 참고 합니다.

* [Convolutional Neural Networks Part 1 - CNN에 대한 전반적인 이해](http://andersonjo.github.io/artificial-intelligence/2016/06/12/Convolutional-Neural-Networks-Part1/)
* [Convolutional Neural Networks Part 1.5 - Convolution연산의 이해](http://andersonjo.github.io/artificial-intelligence/2016/06/12/Convolutional-Neural-Networks-Part1.5/)
* [Convolutional Neural Networks Part 2 - 코드로 구현](http://andersonjo.github.io/artificial-intelligence/2016/06/14/Convolutional-Neural-Networks-Part2/)

위에 저렇게 링크를 걸어놨지만.. 저나 여러분이나 귀찮기 때문에 아마 안볼겁니다. ㅎㅎ <br>
여기서 짧막하게 CNN에 대해서 설명하겠습니다. 

![title](images/cnn.jpg)

이미지가 CNN모델에 입력이 되면 수십~ 수백개의 Kernels (또는 Filters)가 convolution연산을 하게 되고, 이는 feature maps을 생성하게 됩니다. Convolution연산이후 activation 그리고 pooling을 거치게 되며, 위의 그림처럼 여러층에서 convolution + activation + pooling을 거치게 됩니다. Activation은 Non-Linearity를 도입하며, Pooling은 training time을 줄여줍니다. 

특히 Pooling은 이미지의 특정 부분 (region)에 대한 "요약"이라고 볼 수 있으며, 이미지 인식에서 positional (즉 translational) invaiance갖게 됩니다. Invariance란 이미지상의 어떤 물체에 대해서 위치가 바뀌어도 인식을 한다는 뜻입니다. Translation이란 기하학에서 가져온 포인트/픽셀의 위치가 동일한 방향으로 움직이거나 변형되는 것을 말합니다.

![title](images/invariance.png)


## Translation Invariance (Approximately)

### Convolution

$ I $ 는 $ N x N $ 이미지이고, Kernel은 $ M x M $ 이라면 특정위치의 convolution연산은 다음과 같습니다. (p, q는 kernel의 위치, i, j는 이미지에서의 위치)

$$ C_{ij} = \sum_{p,q} I_{i-p, j-q} K_{p, q} $$

이미지를 translate해보도록 합니다. $ k $ position만큼 아래로 내려가고, $ l $ 만큼 오른쪽으로 이동합니다.<br>
즉 $ (i, j) $ 에 위치해있던 픽셀들은 $ (i+k, j+l) $ 만큼 이동을 합니다. <br>
다시 새로운 위치에서  convolution연산을 하면 다음과 같습니다. 

$$ C_{i^{\prime}, j^{\prime}} = \sum_{p, q} I_{i+k-p, j+l-q} K_{p, q} $$

두개의 공식중 달라진 부분은.. $ i^{\prime} = i + k $ 그리고 $ j^{\prime} = j + l $ 입니다.<br>
즉 해당 픽셀들이 translate되어도 동일하게 convolution도 이동을 해서 detect하게 됩니다.

쉽게 말하면 특정 엣지/선등을 kernel이 감지하게 되는데, 이때 해당 엣지/선들이 다른 위치로 옮겨간다 하더라도 <br>
convolution연산에 의해서 위치와는 상관없이 패턴 인식이 가능하게 됩니다.  

### Pooling 

convolution으로 끝나지 않고, Pooling 을 거치면서 실질적인 translation invariance (approximately) 를 줍니다.<br>
예를 들어, Max Pooling의 경우 특정 receptive field안에서 가장 큰 값을 리턴하게 됩니다.<br>
만약 어떤 특정 max값이 오른쪽으로 살짝 이동했지만, 아직 동일한 receptive field안에 존재한다면 max pooling layer는 동일한 값을 리턴하게 될 것입니다.<br>
물론 전제조건은 너무 지나치게 translate되지 않았다는 조건하 입니다. 

> Convolution과 Pooling이 조합되어 CNN은 translational invariance (approximately) 를 갖고 있습니다.

## CNN 의 문제점

위에서 언급했듯이 CNN은 translational invarance (approximately) 이기 때문에 학습시 사용된 이미지와 유사한 이미지를 분류하는것은 정말 굉장히 잘합니다. 하지만 분류하려는 이미지가 rotation, tilt 또는 변형을 가할시 CNN의 performance는 급격하게 떨어지게 됩니다. 
해당 이슈는 학습시에 data augmentation을 통해서 translated된 이미지를 학습함으로서 어느정도 커버를 할 수 있게 됩니다.

아래의 사진과 같은 경우 Data augmentation없이  학습할 경우 인식이 잘 되지 않게 됩니다.

![title](images/invariance2.png)



# Dynamic Routing Between Capsules

인간의 눈은 정말 많은 정보를 얻지만 실제로는 대부분의 정보를 무시합니다. 즉 이 문서를 읽는 동안에도 여기에 써진 글자를 주의깊게 읽고 나머지 주변의 모니터의 제품명, 키보드, 옆에 있는 선풍기, 물컵, 또는 기타등등은 신경쓰지 않습니다. 특히 Fixation point (주의를 깊게 해서 보는 특정 부분)의 경우 말그대로

# References

* [Capsule Network Paper by Hinton](https://arxiv.org/pdf/1710.09829.pdf)
* [What is a CapsNet or Capsule Network?](https://hackernoon.com/what-is-a-capsnet-or-capsule-network-2bfbe48769cc)